In [ ]:
!pip install transformers[torch]

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.c

In [ ]:
from transformers import pipeline

In [ ]:
pipe1  = pipeline(model = "MayankMoon/google-t5-for-summary")
# pipe2 = pipeline(model = "MayankMoon/gemma_for_summary")
pipe3 = pipeline(model = "MayankMoon/output_dir",task="summarization")

In [ ]:
# Sample output of a pipeline
pipe3("The authors present a class of neural process models that are able to produce correlated predictions while amenable to exact, simple and scalable maximum likelihood optimization supporting multiple outputs. By using invertible transformations (gaussian copula), the model is able to capture non-Gaussian output distributions. Experiments with artificial and real data (EEG and climate), highlight the predictive ability of the proposed model. The authors introduce a relatively straightforward extension for Gaussian neural processes in which both mean and (half of the) covariance functions are specified as neural networks, and the covariance function is either explicitly calculated as an inner product in (7) or as a squared exponential covariance function modulated in magnitude by an auxiliary neural network and calculated using the outputs of a neural network rather than the input data itself (x_t, x_c, y_c). Their multi-output and non-Gaussian strategy follows the modulated kernel and Gaussian copula formulation, respectively. Comprehensive experiments on both artificial and real data demonstrate the advantages of the proposed model over the related, but more computationally expensive, fullConvGNP model. Moreover, on real data, the proposed model outperforms both mean field, convNP and MOGP approximations.\n\nSomething that is not discussed in the paper is the setting of the length scale of the squared exponential kernel.\n\nConsidering that one of the motivations of the proposed approach is how prohibitive existing approaches are, having estimates of computational cost and/or runtime experiments could be a welcome addition to the paper. The proposed approach though technically simple relative to existing literature in neural and Gaussian process literature it is well motivated, technically sound and with comprehensive experimental results that support the improved predictive performance claimed by the authors.", max_length = 10000000)


Your max_length is set to 10000000, but your input_length is only 392. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=196)


[{'summary_text': 'The authors present a class of neural process models that are able to produce correlated predictions while amenable to exact, simple and scalable maximum likelihood optimization supporting multiple outputs. The authors introduce a relatively straightforward extension for Gaussian neural processes in which both mean and (half of the) covariance functions are specified as neural networks. The proposed model outperforms both mean field, convNP and MOGP approximations.'}]

In [40]:
# Loading the jsonl file in a list
import json

with open('ORSUM_test.jsonl', 'r') as json_file:
    json_list = list(json_file)

# for json_str in json_list:
#     result = json.loads(json_str)
#     print(f"result: {result}")
#     print(isinstance(result, dict))

In [53]:
data_dict = json.loads(json_list[0])
json_list[393]


'{"Title": "Memory U-Net: Memorizing Where to Vote for Lesion Instance Segmentation", "Authors": ["Hang Zhang", "Jinwei Zhang", "Gufeng Yang", "Pascal Spincemaille", "Thanh D. Nguyen", "Yi Wang"], "Abstract": "Confluent lesions usually occur when pathologically distinct lesions grow close to each other and form a large spatially-connected lesion.   \\nThese confluent lesions pose a great challenge for subsequent image analysis and disease diagnosis, as individual lesions are difficult to separate and segment.\\nIn this paper, we propose a Memory U-Net that takes advantage of recent fully convolutional neural network U-Net and memory networks, to resolve the issue.\\nThe main idea is that we develop a hybrid model with a U-Net for feature extraction and a memory network as the alternative code book for generalized Hough voting.\\nTo alleviate the GPU memory overhead brought by the large code book, we decompose the large code book into three smaller ones, where each one of them accounts 

In [54]:
import pandas as pd

# List to store extracted metareviews
metareviews = []

# Iterate through each JSON string
i = 0
for i in range(len(json_list)-1):
    json_str = json_list[i]
    # Convert JSON string to dictionary

    data_dict = json.loads(json_str)
    # Extract the 'metareview' value and append to the list
    metareview = data_dict.get('Metareview', None)
    if metareview is not None:
        metareviews.append(metareview)

# Create a DataFrame from the extracted metareviews
df_metareviews = pd.DataFrame({'Metareview': metareviews})

# Display the DataFrame
print(df_metareviews)


                                            Metareview
0    This paper proposes a class of neural processe...
1    Exploration can happen at various levels of gr...
2    This paper proposes an unsupervised graph lear...
3    This paper suggests that adversarial vulnerabi...
4    The paper shows that a form of Fictitious Self...
..                                                 ...
388  Reviewers are in agreement that this work is a...
389  The authors design an algorithm for composite ...
390  This paper uses a masked autoencoder as visual...
391  This paper received unanimous recommendations ...
392  This paper presents a model to identify entity...

[393 rows x 1 columns]


In [76]:
SUMMARIZATION_PROMPT ="""
Your task is to generate concise summaries for the metareviews of various papers in the field of [insert specific field, e.g., machine learning].

Instructions:

Read the provided metareview carefully.
Use the information from the metareview to generate a summary that captures the key points and findings of the paper's review process.
Ensure that the summary is concise, informative, and accurately represents the content of the metareview.
Your summary should be between 2 to 4 sentences long.
Focus on highlighting the main contributions, strengths, weaknesses, and overall recommendation of the paper as evaluated by the reviewers.
Example Metareview:

Metareview: "This paper proposes a novel approach to neural network training, leveraging recent advancements in optimization techniques. Reviewers found the experimental results compelling, demonstrating significant improvements over existing methods. However, some concerns were raised regarding the generalization of the approach to different datasets. Overall, the paper was recommended for acceptance with minor revisions."

Sample Summary:

Summary: "The paper introduces a novel neural network training approach, showing promising results in experiments. Reviewers highlighted its potential for significant advancements in the field, although some concerns about generalization were noted. Overall, the paper received a positive recommendation with minor revisions."

Your Task:

Using the information provided in the metareview dataframe, generate summaries for each of following metareview:
{metareview}"""

In [77]:
pipe3(SUMMARIZATION_PROMPT)

[{'summary_text': 'Your task is to generate summaries for the metareviews of various papers in the field of [insert specific field, e.g., machine learning]. Use the information provided in the dataframe to generate a summary that captures the key points and findings of the review process. Ensure that the summary is concise, informative, and accurately represents the content of the paper. Reviewers found the experimental results compelling, demonstrating significant improvements over existing methods. Overall, the paper received a positive recommendation with minor revisions.'}]

In [73]:
summary = []
for i in range(len(df_metareviews)):
  input = df_metareviews.Metareview[i]
  summary.append(pipe3(SUMMARIZATION_PROMPT.format(metareview=input))[0]["summary_text"])
  print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [74]:
summary

['This paper proposes a class of neural processes that lifts the limitations of conditional neural processes (CNPs) and produces dependent/correlated outputs but that, as CNPs, is inherently scalable and it is easy to train via maximum likelihood. The proposed model is extended to multi-output regression and to capture non-Gaussian output distributions. The paper parameterizes the prediction map as a Gaussian, where the mean and covariance are determined using neural networks.',
 'This paper introduces a novel approach to neural network training, leveraging recent advancements in optimization techniques. Reviewers found the experimental results compelling, demonstrating significant improvements over existing methods. Overall, the paper received a positive recommendation with minor revisions. The reviewers agree that this paper opens up and tackles rebuttal, and provides an extensive empirical study on atari games (a standard benchmark for such problem settings). While the reviewers agr

In [75]:
  # input = df_metareviews.Metareview[0]
  # pipe3(SUMMARIZATION_PROMPT.format(metareview=input))[0]["summary_text"]

In [67]:
# input = df_metareviews.Metareview[0]
# input

'This paper proposes a class of neural processes that lifts the limitations of conditional neural processes (CNPs) and produces dependent/correlated outputs but that, as CNPs, is inherently scalable and it is easy to train via maximum likelihood. The proposed model is extended to multi-output regression and to capture non-Gaussian output distributions. Results are presented on synthetic data, an electroencephalogram dataset and on a climate modeling problem. The paper parameterizes the prediction map as a Gaussian, where the mean and covariance are determined using neural networks. Non-Gaussian prediction maps are obtained using copulas. \n\nTechnically speaking, the reviewers found the approach to be incremental and only marginally significant and I agree with them. Issues such as estimates of computational cost, using fixed lengthscales for the covariances and relationships/using normalizing flows have been addressed by the authors satisfactorily. Empirically, the contribution of the

In [69]:
# SUMMARIZATION_PROMPT.format(metareview=input)

'\nYour task is to generate concise summaries for the metareviews of various papers in the field of [insert specific field, e.g., machine learning].\n\nInstructions:\n\nRead the provided metareview carefully.\nUse the information from the metareview to generate a summary that captures the key points and findings of the paper\'s review process.\nEnsure that the summary is concise, informative, and accurately represents the content of the metareview.\nYour summary should be between 2 to 4 sentences long.\nFocus on highlighting the main contributions, strengths, weaknesses, and overall recommendation of the paper as evaluated by the reviewers.\nExample Metareview:\n\nMetareview: "This paper proposes a novel approach to neural network training, leveraging recent advancements in optimization techniques. Reviewers found the experimental results compelling, demonstrating significant improvements over existing methods. However, some concerns were raised regarding the generalization of the appr

In [71]:
# pipe3(SUMMARIZATION_PROMPT.format(metareview=input))[0]["summary_text"]

'This paper proposes a class of neural processes that lifts the limitations of conditional neural processes (CNPs) and produces dependent/correlated outputs but that, as CNPs, is inherently scalable and it is easy to train via maximum likelihood. The proposed model is extended to multi-output regression and to capture non-Gaussian output distributions. The paper parameterizes the prediction map as a Gaussian, where the mean and covariance are determined using neural networks.'